In [11]:
from osgeo import gdal, ogr, osr
import matplotlib.pyplot as plt
import glob
import geopandas as gpd
import numpy as np

In [12]:
#reading classified images from 2002-2022

folderPath="C:\\Users\\Sreehari\\Documents\\IIRS\\BigGeoProj\\Results"


clsImgs=[]
noData=[]
fileList=sorted(glob.glob(folderPath+'/*.tif'))
for filePath in fileList:
    ds=gdal.Open(filePath)
    noData.append(ds.GetRasterBand(1).GetNoDataValue())
    clsImgs.append(ds)
    ds=None

In [13]:
#finding the area of builtup in each image

img1=clsImgs[0].ReadAsArray()
img2=clsImgs[1].ReadAsArray()
img3=clsImgs[2].ReadAsArray()

In [14]:
lulcNo=0
pixelSize= 30*30


mask1 = img1 == lulcNo
area1=np.sum(mask1)*pixelSize

mask2= img2 == lulcNo
area2=np.sum(mask2)*pixelSize

mask3= img3 == lulcNo
area3= np.sum(mask3)*pixelSize

print("Area of Builtup in 2000 (km2): ",area1/1e+6)
print("Area of Builtup in 2012 (km2): ",area2/1e+6)
print("Area of Builtup in 2022 (km2): ",area3/1e+6)


Area of Builtup in 2000 (km2):  289.8648
Area of Builtup in 2012 (km2):  381.8034
Area of Builtup in 2022 (km2):  513.8784


In [ ]:
count=np.count_nonzero(~np.isnan(img3))
totArea=count*pixelSize
print(totArea)

In [ ]:
#finding Annual Change Intensity ACI

LC= area3-area1 #area of land change from non built to built
LA= totArea #area of entire landscape
TE = 20 #years (duration of given time interval)

ACI = ((LC/LA)/TE)*100
print("Annual Change Intensity: ", ACI)

In [ ]:
img1=img1+1
img2=img2+1
img3=img3+1

In [ ]:
#total change
change=(img1*10)+img3
np.unique(change)

In [ ]:
change=np.nan_to_num(change,nan=0)
change.shape

In [ ]:
#creating change matrix
changeMatrix=np.zeros((4,4))
for p in range(0,change.shape[0]):
    for q in range(0,change.shape[1]):
        a=change[p,q]
        i=int(a/10)
        j=int(a%10)
        changeMatrix[i-1,j-1]+=1
        
        
        

In [ ]:
#printing change matrix by calculating area
changeMatrix=(changeMatrix*pixelSize)/1e+6
print(changeMatrix)  

In [ ]:
#converting change matrix to dataframe

cmDf= pd.DataFrame(data=changeMatrix, columns=['1','2','3','4'])
cmDf.insert(loc=0, column='Classes', value=['Built-up','Barren','Vegetation','Waterbodies'])
cmDf.insert(loc=1,column='',value=['1','2','3','4'])
cmDf.to_csv("C:\\Users\\Sreehari\\Documents\\IIRS\\BigGeoProj\\Results\\change.csv", sep=',')
cmDf.head()